<a href="https://colab.research.google.com/github/CutieeYummyy/DATAANALYSISDEEPLEARNING/blob/main/TH%E1%BB%B0C_H%C3%80NH_BU%E1%BB%94I_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Default title text
import pandas as pd
import numpy as np
#Problem 1
# Thêm header vào DataFrame để diễn giải dữ liệu
column_names = ['ID', 'Name', 'Age', 'Weight', 'm0006', 'm0612', 'm1218', 'f0006', 'f0612', 'f1218']

# Read the CSV file
df = pd.read_csv(r'/patient_heart_rate.csv', names=column_names, on_bad_lines='skip', skiprows=1)

# Display the first few rows of the dataframe
print(df.head())

#Problem 2
df[["FirstName", "LastName"]] = df["Name"].str.split(" ", n=1, expand=True)
df= df.drop('Name', axis=1)
#print(df)

#Problem 3
#Get the Weight column
weight = df["Weight"]
def convert_weight(weight):
    if isinstance(weight, str):
        if 'lbs' in weight:
            weight = weight[:-3]
            return str(int(float(weight) / 2.2)) + 'kgs'
    return weight

# Áp dụng hàm chuyển đổi cho cột 'Weight' trên bản sao DataFrame
df["Weight"] = weight.apply(convert_weight)
#print(df)

#Problem 4
df.dropna(how='all', inplace=True)
#print(df)

#Problem 5
df=df.drop_duplicates(subset=["FirstName","LastName","Age","Weight"])
#print(df)

#Problem 6
df.FirstName.replace({r'[\x00-\x7F]+':''}, regex=True, inplace=True)
df.LastName.replace({r'[\x00-\x7F]+':''}, regex=True, inplace=True)
#print(df)

#Problem 7
#Deletion: Remove records with missing values
df = df.dropna(subset=['Age', 'Weight'])
#print("\nDataFrame after Deletion:")
#print(df.head())
#Dummy substitution: Replace missing values with a dummy but valid value:e.g.: 0 for numerical values
df_dummy = df.fillna({'Age': 0, 'Weight': '0kgs'})
#print("\nDataFrame after Dummy substitution:")
#print(df_dummy.head())
#Mean substitution: Replace the missing values with the mean
df_mean = df.copy()
df_mean['Age'] = df_mean['Age'].fillna(df_mean['Age'].mean())
df_mean['Weight'] = df_mean['Weight'].fillna(df_mean['Weight'].apply(lambda x: int(x[:-3]) if isinstance(x, str) else x).mean())
df_mean['Weight'] = df_mean['Weight'].astype(str) + 'kgs'
#print("\nDataFrame after Mean substitution:")
#print(df_mean.head())
#Frequent substitution: Replace the missing values with the most frequent item.
df_frequent = df.copy()
df_frequent['Age'] = df_frequent['Age'].fillna(df_frequent['Age'].mode()[0])
df_frequent['Weight'] = df_frequent['Weight'].fillna(df_frequent['Weight'].mode()[0])
#print("\nDataFrame after Frequent substitution:")
#print(df_frequent.head())
#Yêu cầu
# Thống kê thông tin dữ liệu thiếu
missing_info = df[['Age', 'Weight']].isnull().sum()
missing_info_percentage = df[['Age', 'Weight']].isnull().mean() * 100

#print("Missing values count:\n", missing_info)
#print("Missing values percentage:\n", missing_info_percentage)

#Problem 8
#Mekt the Sex+ time range column in single column
df = pd.melt(df, id_vars=['ID','Age','Weight', 'FirstName', 'LastName'], var_name='sex_and_time').sort_values(['ID','Age','Weight','FirstName','LastName'])
#Extract Sex, Hour Lower bound and Hour upper bound group
tmp_df = df["sex_and_time"].str.extract(r"(\D)(\d+)(\d{2})", expand=True) # Added opening parenthesis before \D to balance the expression
#Name columns
tmp_df.columns = ['Sex', 'Hour_Lower', 'Hour_Upper']
#Create Time column based in 'Hour_Lower' and "hour_Upper" columns
tmp_df["Time"]=tmp_df["Hour_Lower"]+ "-" +tmp_df["Hour_Upper"]
#merge
df = pd.concat([df, tmp_df], axis=1)
#Drop unneccessary columns and rows
df = df.drop(['sex_and_time','Hour_Lower','Hour_Upper'], axis=1)
df = df.dropna()
df.to_csv('outputcleanup.csv', index=False)
#print(df)

#Problem 9

# Calculate the missing data percentage on blood pressure columns
missing_bp = df[['m0006', 'm0612', 'm1218', 'f0006', 'f0612', 'f1218']].isnull().mean() * 100
print("Missing data percentage on blood pressure columns:")
print(missing_bp)

# Function to replace missing blood pressure values

def replace_missing_bp(df):
    for col in ['m0006', 'm0612', 'm1218', 'f0006', 'f0612', 'f1218']:
        for i in range(len(df)):
            if pd.isna(df.loc[i, col]):
                replaced = False

                # 1) Replace with average of immediate neighbors
                if i > 0 and i < len(df) - 1 and pd.notnull(df.loc[i-1, col]) and pd.notnull(df.loc[i+1, col]):
                    neighbor1 = float(df.loc[i-1, col]) if pd.notnull(df.loc[i-1, col]) and isinstance(df.loc[i-1, col], str) else df.loc[i-1, col]
                    neighbor2 = float(df.loc[i+1, col]) if pd.notnull(df.loc[i+1, col]) and isinstance(df.loc[i+1, col], str) else df.loc[i+1, col]
                    df.loc[i, col] = (neighbor1 + neighbor2) / 2
                    replaced = True

                # 2) Replace with average of two previous values
                if not replaced and i > 1 and pd.notnull(df.loc[i-1, col]) and pd.notnull(df.loc[i-2, col]):
                    prev1 = float(df.loc[i-1, col]) if pd.notnull(df.loc[i-1, col]) and isinstance(df.loc[i-1, col], str) else df.loc[i-1, col]
                    prev2 = float(df.loc[i-2, col]) if pd.notnull(df.loc[i-2, col]) and isinstance(df.loc[i-2, col], str) else df.loc[i-2, col]
                    df.loc[i, col] = (prev1 + prev2) / 2
                    replaced = True

                # 3) Replace with average of two subsequent values
                if not replaced and i < len(df) - 2 and pd.notnull(df.loc[i+1, col]) and pd.notnull(df.loc[i+2, col]):
                    next1 = float(df.loc[i+1, col]) if pd.notnull(df.loc[i+1, col]) and isinstance(df.loc[i+1, col], str) else df.loc[i+1, col]
                    next2 = float(df.loc[i+2, col]) if pd.notnull(df.loc[i+2, col]) and isinstance(df.loc[i+2, col], str) else df.loc[i+2, col]
                    df.loc[i, col] = (next1 + next2) / 2
                    replaced = True
                # 4) Replace with average of all values for that person
                if not replaced:
                    person_bp_values = df.loc[i, ['m0006', 'm0612', 'm1218', 'f0006', 'f0612', 'f1218']].dropna().astype(float)
                    if len(person_bp_values) > 0:
                        df.loc[i, col] = person_bp_values.mean()
                    else:
                        # 5) Replace with average of all values for that gender group
                        gender_group = col[0]
                        gender_bp_values = df[df['gender'] == gender_group][['m0006', 'm0612', 'm1218', 'f0006', 'f0612', 'f1218']].stack().astype(float)
                        if len(gender_bp_values) > 0:
                            df.loc[i, col] = gender_bp_values.mean()
                        else:
                            # 6) Replace with overall average
                            overall_bp_values = df[['m0006', 'm0612', 'm1218', 'f0006', 'f0612', 'f1218']].stack().astype(float)
                            df.loc[i, col] = overall_bp_values.mean()


    return df

# Apply the function to replace missing blood pressure values
df = replace_missing_bp(df)

# Display the dataframe after replacing missing blood pressure values
print("\nDataFrame after replacing missing blood pressure values:")
print(df.head())

# Problem 10

# Drop any rows with missing values in the dataset
df_clean = df.dropna()

# Reindex the cleaned dataframe
df_clean.reset_index(drop=True, inplace=True)

# Save the cleaned data to a CSV file
df_clean.to_csv('patient_data_clean.csv', index=False)

print("Cleaned data saved to patient_data_clean.csv successfully.")







    ID            Name   Age      Weight m0006 m0612 m1218 f0006 f0612 f1218
0  2.0     Donald Duck  34.0   154.89lbs     -     -     -    85    84    76
1  3.0      Mini Mouse  16.0         NaN     -     -     -    65    69    72
2  4.0  Scrooge McDuck   NaN       78kgs    78    79    72     -     -     -
3  5.0    Pink Panther  54.0  198.658lbs     -     -     -    69   NaN    75
4  6.0     Huey McDuck  52.0      189lbs     -     -     -    68    75    72


KeyError: "None of [Index(['m0006', 'm0612', 'm1218', 'f0006', 'f0612', 'f1218'], dtype='object')] are in the [columns]"